In [1]:
## import the objects from the sora package
from sora import Star, Observer, Ephemeris, Occultation
import astropy.units as u

ImportError: cannot import name 'Ephemeris' from 'sora' (/home/rcboufleur/Projects/SORA/sora/__init__.py)

#### defining star

In [ ]:
star = Star(ra='13 50 45', dec='-23 14 21', distance=30*u.pc)
star.apparent_radius(distance=10*u.AU) # it will search for the star radius on the gaia catalogue or the magnitudes in some catalogue to calculate distance

star2 = Star(gaia_id='546164465168') # it will search the star in the gaia catalogue and get all the informations needed

#### defining observations

In [ ]:
lna = Observer(874) # and it searchs in a database the location
opd = Observer(lon='-45 34 57', lat='-22 32 04', height=1864)

In [ ]:
lna.fit_curve('light_curve.txt', star=star) # and it saves the curve, and calculates the immersion and emersion, and also:
lna.fit_curve('light_curve.txt', 'negative') # in this case it does not fit, but saves the curve and gets start and final instants

# or we can define the instants of immersion and emersion directly
lna.immersion = Time('2019-12-06 13:00:00')
lna.emersion = Time('2019-12-06 13:01:00')
opd.obs_begin = Time('2019-12-06 12:50:00') # in case of negative
opd.obs_end = Time('2019-12-06 13:20:00') # in case of negative

# maybe this can be called inside the Occultation object because it fits the shape, calculate the offset,
# then repeat the fit of the curve with a new occultation velocity relative to the limb

#### defining ephemeris

In [ ]:
ephem = Ephemeris('phoebe_sat375.txt') # reads files as the ephem_planete does
#or
ephem = Ephemeris('Phoebe', begin='2019-12-06 00:00:00', end='2019-12-07 00:00:00') # and it downloads the ephemeris from JPL

In [ ]:
ephem.add_offset(10*u.mas, 8*u.mas)
t = Time('2019-12-06 00:00:00', scale='utc')
n_pos = ephem.get_position(t) # it calculates the function of ksi_eta if it hasn't
n_pos = ephem.get_position('2019-12-06 00:00:00') # in this case I suggest to define a default in UTC or TDB, probably UTC
ksi,eta = ephem.get_ksi_eta(t, star) #returns the ksi, eta

#### Fazendo uma redução

In [ ]:
reduc = Occultation(star, ephem)
#or
reduc = Occultation(star='13 50 45 -23 14 21', ephem='phoebe_sat375.txt')
# in this case, it will automatically call Star and Ephemeris internally
# I don't prefer this because we don't have direct access to the Star and Ephemeris objects.

In [ ]:
# adding observations
reduc.add_observation(lna) # the input is the object defined above
reduc.add_observation(opd)
reduc.add_observation('511') # in this case, it will automatically call Observer internally
# This creates a list in reduc with all observations added.

In [ ]:
reduc.fit_ellipse()
#or
reduc.fit_to_shape('3d_file.obj')
# fit to a 3D shape model, but maybe in this case it would be interesting to have the 3d model as another object.

After doing this, let's say I want to test this reduction with another ephemeris, I could do

In [ ]:
ephem2 = Ephemeris('phoebe_josselin.txt')
reduc.ephem = ephem2
reduc.fit_ellipse()
# I don't need to add the observations or the star again

# Probably, if I just do
ephem = Ephemeris('phoebe_josselin.txt')
# I don't need to inform reduc, because reduc already has a pointer to the "ephem" variable defined above, but I'm not sure.

Now, let's say I made an error in the longitude of the OPD, I can just

In [ ]:
opd.lon = '-45 35 57'
# since I already added opd to reduc, reduc will be updated as well.

Of course, all this example is not exact. We may change the name of the variables and methods of the object, and how they are accessed.
But this gives us hints on how the code can be used.